# LLM推理框架SGLang和Vllm使用  


# Vllm
> https://docs.vllm.ai/en/latest/api/index.html#model-development
## 基本使用
使用vllm进行模型推理过程比较简单直接按照下面大致模板即可：
```python
import time
from vllm import LLM, SamplingParams
def load_model(model_name, cache_dir=None):
    llm = LLM(
        model=model_name,
        trust_remote_code=True,
        dtype="bfloat16",             # 或 "float16"
        tensor_parallel_size=1,       # 多GPU时可>1
        download_dir=cache_dir        # 指定缓存路径
    )
    return llm

def model_generate(llm, prompt):
    s_time = time.time()
    sampling_params = SamplingParams(
        temperature=0.8,          # 控制随机性
        top_p=0.9,                # nucleus sampling
        top_k=40,                 # 限制采样候选
        repetition_penalty=1.05,  # 防止复读
        max_tokens=128,           # 最大生成长度
        logprobs=5,               # 输出 top-5 token 概率
    )
    outputs = llm.generate([prompt], sampling_params)
    print(f"✅ 推理完成，用时：{time.time()- s_time:.2f} 秒")
```
通过上面的方式就可以实现：加载模型+使用模型进行推理（通过`SamplingParams`来控制生成内容）上面例子过程中各个部分解释如下：  
1、`LLM`（实际过程中可以使用参数不只是上面提到的几种，比如说在[GRPO-vllm used](https://github.com/huggingface/trl/blob/b8f23ef3bd8d7d4fbd5b533de4e7e8c50103231f/trl/trainer/grpo_trainer.py#L560) 使用到的参数就很多，具体的[模型参数描述](https://docs.vllm.ai/en/latest/api/vllm/index.html#vllm.LLM)）：首先、是一些**基础的模型参数**：`model`，`download_dir`，`dtype`这几个参数主要是是要用什么模型下载在那个文件夹模型的权重是什么。
- `tensor_parallel_size`：张量并行数量（单卡：1多卡就可以直接根据卡去设定数量）；
- `gpu_memory_utilization`：控制 vLLM 占用 GPU 显存的比例
- `max_num_seqs`：控制 KV cache 预热时允许同时处理的序列数量。
- `max_model_len`：模型可支持的最大序列长度（决定KV cache预分配的大小：$KV\_cache\_size≈max\_num\_seqs×max\_model\_len×hidden\_dim$）
- `max_num_batched_tokens`：控制 vLLM 批处理时允许的 token 数量上限
- `logprobs_mode`：控制 vLLM 输出 token 的 logprob 类型。"processed_logprobs" → 经过温度缩放、logit 调整后的最终 logprob。"raw_logprobs" 会返回未处理的原始 logit 概率。
- `enable_sleep_mode`: 启用 sleep mode，在无请求时让 EngineCore 休眠  

2、`generate`过程：直接使用`llm.generate([prompt], sampling_params)`就可以得到输出，其中模型输出构成为（[CompletionOutput](https://docs.vllm.ai/en/latest/api/vllm/index.html#vllm.CompletionOutput)）：
```python
outputs (list of GenerationResult)  # 长度 = 输入 prompt 数量
│
├─ outputs[0] (GenerationResult for prompt 0)
│   ├─ prompt: "写一首七言绝句"
│   ├─ request_id: "xxxx"
│   └─ outputs (list of GenerationOutput)  # 长度 = SamplingParams.n
│       ├─ outputs[0] (GenerationOutput)
│       │   ├─ text: "xxxx....."
│       │   ├─ token_ids: [358, 1184, 311, ....]
│       │   ├─ logprobs: [{...}, ...]
│       │   ├─ finish_reason: "eos_token"
│       │   └─ n_generated_tokens: 32
│       ├─ outputs[1] (GenerationOutput)
│       │   ├─ text: ...
│       │   └─ ...
│       └─ outputs[2] ...
│
├─ outputs[1] (GenerationResult for prompt 1)
│   └─ outputs[0] ...
└─ outputs[2] ...
```
值得注意的是`logprobs`代表的是每个词的生成概率，比如说结果如下：
```python
[{358: Logprob(logprob=-1.6076639890670776, rank=1, decoded_token=' I'), 
       4710: Logprob(logprob=-1.6857889890670776, rank=2, decoded_token=' \n\n'), 
       481: Logprob(logprob=-2.154539108276367, rank=3, decoded_token=' -'), 
       576: Logprob(logprob=-2.935789108276367, rank=4, decoded_token=' The'), 
       5209: Logprob(logprob=-3.170164108276367, rank=5, decoded_token=' Please')}, 
       
       {1184: Logprob(logprob=-1.0075775384902954, rank=1, decoded_token=' need'), 
        2776: Logprob(logprob=-1.4763275384902954, rank=2, decoded_token="'m"), 
        1079: Logprob(logprob=-2.101327419281006, rank=3, decoded_token=' am'), 
        614: Logprob(logprob=-2.257577419281006, rank=4, decoded_token=' have'), 
        1366: Logprob(logprob=-2.648202419281006, rank=5, decoded_token=' want')},
...]
```
里面list中的每一个字典都对应第i个词每个词的概率，比如说第一个词输出 `I`的概率最大依次类推。
## 后端推理
代码不复杂只是涉及到更加多的是后端代码的书写（`./vllm_server.py`）下面就是简单调用的测试例子：
```python
import pprint
import requests
resp = requests.post(
    "http://127.0.0.1:8000/generate",
    json={
        "prompt": "写一首关于秋天的诗",
        "max_tokens": 128,
        "temperature": 0.7,
        "top_p": 0.9,
        "n": 2
    }
)
pprint.pprint(resp.json())
```

In [9]:
import pprint
import requests
resp = requests.post(
    "http://127.0.0.1:8000/generate",
    json={
        "prompt": "写一首关于秋天的诗",
        "max_tokens": 1024,
        "temperature": 0.7,
        "top_p": 0.9,
        "n": 2
    }
)
pprint.pprint(resp.json())

{'results': ['，要求内容不超极限，用词要符合秋天的意象，比如落叶、银杏、枫叶、稻谷、丰收，但不要写成一首诗，而是用文字描述，描述秋天的景象。注意不能使用任何标点，用汉字组成句子，每句七个字左右，结构对称，押韵，每句结尾押韵，开头是动词，结尾是动词，四字成语或四字短语。\n'
             '题目要求是“秋天的景色”。\n'
             '好的，我需要完成一首关于秋天的诗，但不要写成诗，而是用文字描述。首先，我得确定秋天的典型意象，比如落叶、银杏、枫叶、稻谷、丰收。这些元素都是秋天的典型特征，但要注意不能超过极限，所以得简洁。\n'
             '\n'
             '接下来，结构方面，题目要求四字成语或四字短语，每句七个字，结构对称，押韵，开头和结尾都是动词。可能需要先列出一些可能的句子，然后调整结构。\n'
             '\n'
             '比如，开头用动词开头，比如“金黄铺展”，然后后面接描述。然后结尾也要动词结尾，比如“归雁归心”。这样结构对称，押韵。\n'
             '\n'
             '比如第一句：“金黄铺展”，第二句“银杏叶落”，第三句“稻穗低垂”，第四句“丰收满园”。这样结构对称，每句七个字，押韵，动词开头和结尾。不过可能需要调整用词，使更符合秋天的意象。\n'
             '\n'
             '或者更具体一些，比如“枫叶染红”，“稻谷沉香”，“银杏摇落”，“归雁南飞”。这样更生动，符合秋天的景象。\n'
             '\n'
             '然后检查是否符合四字结构，每句七个字，开头和结尾动词，四字短语。可能需要多次调整，确保没有标点，用汉字组成句子。\n'
             '\n'
             '比如：\n'
             '\n'
             '金黄铺展，银杏叶落。稻穗低垂，丰收满园。枫叶染红，稻谷沉香。归雁南飞，秋意浓。\n'
             '\n'
             '这样每句七个字，结构对称，开头和结尾动词，四字短语，押韵。可能还可以再调整，比如“枫叶红，稻谷香，银杏落，归雁飞”这样更押韵